In [79]:
import panel as pn
import mricloudpy as mp
import imaging
from tkinter import Tk, filedialog
import ctypes
import plotly.io as pio
pio.renderers.default = 'browser'

pn.extension('plotly')
# pn.extension('tabulator')

# Kermit, Miss Piggy, Fozzie, Gonzo, Rowlf, Scooter, Animal, Pepe, Rizzo, Beaker, Statler, Waldorf, Swedish Chef

### Sidebar items

In [80]:
dir_select_button = pn.widgets.Button(name='Select data directory', button_type='primary')
dir_select_text = pn.widgets.StaticText(visible=False)
id_type_select = pn.widgets.Select(name='ID Type', options={'Numeric': 'numeric',
                                                         'Filename': 'filename',
                                                         'Custom': 'custom'})
id_list_text = pn.widgets.TextAreaInput(name='ID List', visible=False, 
                                        placeholder='ex: Kermit, Miss Piggy, Fozzie...')
file_upload_button = pn.widgets.Button(name='Upload', button_type='primary')
data = pn.widgets.Tabulator(height=500, width=300, page_size=100, pagination='local', disabled=True)

### Main items

In [81]:
vis_select = pn.widgets.Select(name='Visual', options={'Sunburst':'sunburst', 
                                                       'Treemap': 'treemap',
                                                       'Icicle': 'icicle',
                                                       'Bar': 'bar',
                                                       'Mean Difference': 'mean_diff',
                                                       'Correlation Matrix': 'corr_matrix',
                                                       'Neuroimaging': 'imaging'},
                                                       width=140)
type_select = pn.widgets.Select(name='Type', options=['1', '2'], width=50, visible=True)
base_level_select = pn.widgets.Select(name='Base Level', options=['1', '2', '3', '4', '5'], width=50, visible=True)
id_single_select = pn.widgets.Select(name='ID', options=[], width=100, visible=True)
id_multi_text = pn.widgets.TextAreaInput(name='ID List', placeholder='ex: Kermit, Miss Piggy... (or \"all\")',
                                          width=150, visible=False)
x_select = pn.widgets.Select(name='x', options=['ID', 'Object'], width=100, visible=False)
y_select = pn.widgets.Select(name='y', options=['Prop', 'Volume'], width=100, visible=False)
log_select = pn.widgets.Select(name='Logarithm', options=['False', 'True'], width=100, visible=False)
image_upload_button = pn.widgets.Button(name='Upload image file', button_type='primary', margin=(35, 10, 10, 10), visible=False)
image_select_text = pn.widgets.StaticText(width=100, margin=(30, 10, 10, 10), visible=False)
image_view_select = pn.widgets.Select(name='View', options={'Horizontal': 0, 
                                                       'Sagittal': 1,
                                                       'Coronal': 2},
                                                       width=100, visible=False)
image_size_input = pn.widgets.IntInput(name='Row/Column Size', value=1, step=1, start=1, end=10, width=100, visible=False)
image_size_tooltip = pn.widgets.TooltipIcon(value='Select size \'1\' to view individual slices. ' + 
                                            'Select a size greater than \'2\' to generate a grid of subplots.',
                                            visible=False)
region_selector = pn.widgets.CrossSelector(name='Regions', options=[], visible=False)
generate_button = pn.widgets.Button(name='Generate', button_type='primary')

vis_plot = pn.pane.Plotly(sizing_mode="stretch_width", config={'responsive': True}, height=700, visible=False)
image_plot = pn.pane.Plotly(height=800, width=800, visible=False)

# Imaging module navigation arrows and input
image_plot_int_input = pn.widgets.IntInput(start=0, width=100)
image_plot_int_input_button = pn.widgets.Button(name='Jump to Slice', button_type='primary', width=50)
image_plot_left_arrow_single = pn.widgets.Button(icon='chevron-left', button_type='default', width=50)
image_plot_right_arrow_single = pn.widgets.Button(icon='chevron-right', button_type='default', width=50)
image_plot_left_arrow_multiple = pn.widgets.Button(icon='chevrons-left', button_type='default', width=50)
image_plot_right_arrow_multiple = pn.widgets.Button(icon='chevrons-right', button_type='default', width=50)

# slider = pn.widgets.IntSlider(name='Integer Slider', start=0, end=100, step=0, value=50)


### Sidebar events

In [82]:
def dir_select(event):
    ctypes.windll.shcore.SetProcessDpiAwareness(1)
    root = Tk()
    root.withdraw()                                        
    root.call('wm', 'attributes', '.', '-topmost', True) 
    # root.call('tk', 'scaling', 2.0)  
    dir_select.dir = filedialog.askdirectory(mustexist=True)
    dir_select_text.value = dir_select.dir
    dir_select_text.visible = True

def check_custom(event):
    if event.new == 'custom':
        id_list_text.visible = True
    else:
        id_list_text.visible = False

def custom_list_format(event):
    text = event.new
    text = text.replace(', ', ',')
    custom_list_format.text_list = text.split(',')

def upload(event):
    upload.is_uploaded = False
    if dir_select.dir is not None:
        try:
            upload.dataset = mp.Data(path=dir_select.dir, id_type=id_type_select.value, 
                             id_list=custom_list_format.text_list)
            data.value = upload.dataset.get_data()
        except:
            upload.dataset = mp.Data(path=dir_select.dir, id_type=id_type_select.value)
            data.value = upload.dataset.get_data()
        data.visible = True
        id_single_select.options = data.value['ID'].unique().tolist()
        upload.is_uploaded = True

dir_select_button.on_click(dir_select)
id_type_select.param.watch(check_custom, 'value')
id_list_text.param.watch(custom_list_format, 'value')
file_upload_button.on_click(upload)

data.visible = False

side_col = pn.Column('# Upload Data',
                     dir_select_button,
                     dir_select_text, 
                     id_type_select,
                     id_list_text, 
                     file_upload_button,
                     pn.layout.Divider(),
                     '### Data Preview',
                     data)

### Main events

In [83]:
def check_vis_type(event):
    if event.new == 'sunburst' or event.new == 'treemap' or event.new == 'icicle':
        type_select.visible = True
        base_level_select.visible = True
        id_single_select.visible = True
        id_multi_text.visible = False
        x_select.visible = False
        y_select.visible = False
        log_select.visible = False
        
        # Image upload
        image_upload_button.visible = False
        image_select_text.visible = False
        # Image options row
        image_view_select.visible = False
        image_size_input.visible = False
        image_size_tooltip.visible = False
        # Image region selector
        region_selector.visible = False
        # Image navigation row
        image_plot_arrow_row.visible = False
        image_plot.visible = False
    elif event.new == 'bar':
        type_select.visible = True
        base_level_select.visible = True
        id_single_select.visible = False
        id_multi_text.visible = True
        x_select.visible = True
        y_select.visible = True
        log_select.visible = True

        # Image upload
        image_upload_button.visible = False
        image_select_text.visible = False
        # Image options row
        image_view_select.visible = False
        image_size_input.visible = False
        image_size_tooltip.visible = False
        # Image region selector
        region_selector.visible = False
        # Image navigation row
        image_plot_arrow_row.visible = False
        image_plot.visible = False
    elif event.new == 'mean_diff':
        type_select.visible = True
        base_level_select.visible = True
        id_single_select.visible = False
        id_multi_text.visible = True
        x_select.visible = True
        y_select.visible = False
        log_select.visible = False
        
        # Image upload
        image_upload_button.visible = False
        image_select_text.visible = False
        # Image options row
        image_view_select.visible = False
        image_size_input.visible = False
        image_size_tooltip.visible = False
        # Image region selector
        region_selector.visible = False
        # Image navigation row
        image_plot_arrow_row.visible = False
        image_plot.visible = False
    elif event.new == 'corr_matrix':
        type_select.visible = True
        base_level_select.visible = True
        id_single_select.visible = False
        id_multi_text.visible = True
        x_select.visible = False
        y_select.visible = False
        log_select.visible = False
        
        # Image upload
        image_upload_button.visible = False
        image_select_text.visible = False
        # Image options row
        image_view_select.visible = False
        image_size_input.visible = False
        image_size_tooltip.visible = False
        # Image region selector
        region_selector.visible = False
        # Image navigation row
        image_plot_arrow_row.visible = False
        image_plot.visible = False
    elif event.new == 'imaging':
        type_select.visible = False
        base_level_select.visible = False
        id_single_select.visible = False
        id_multi_text.visible = False
        x_select.visible = False
        y_select.visible = False
        log_select.visible = False
        
        # Image upload
        image_upload_button.visible = True
        image_select_text.visible = False
        # Image options row
        image_view_select.visible = False
        image_size_input.visible = False
        image_size_tooltip.visible = False
        # Image region selector
        region_selector.visible = False
        # Image navigation row
        image_plot_arrow_row.visible = False
        vis_plot.visible = False
        
def id_list_format(event):
    text = event.new
    if text == 'all':
        id_list_format.text_list = list(id_single_select.options)
    else:
        text = text.replace(', ', ',')
        id_list_format.text_list = text.split(',')
    

def generate(event):
    if upload_image.image_is_uploaded:
        if vis_select.value == 'imaging':
            image_plot.object = imaging.generate_3d_image(img_path=image_select_text.value, 
                                                        regions=region_selector.value,
                                                        view=image_view_select.value, 
                                                        nrows=image_size_input.value,
                                                        ncols=image_size_input.value,
                                                        slice_n=0)
            image_plot_int_input.end = 100
            if image_size_input.value > 1:
                image_plot.visible = False
            else:
                image_plot.visible = True
                image_plot_arrow_row.visible = True
                
        
    if upload.is_uploaded:
        if vis_select.value == 'sunburst':
            vis_plot.object = upload.dataset.generate_sunburst(type=int(type_select.value), 
                                                        id=str(id_single_select.value), 
                                                        base_level=int(base_level_select.value))
            vis_plot.visible = True
        elif vis_select.value == 'treemap':
            vis_plot.object = upload.dataset.generate_treemap(type=int(type_select.value), 
                                                        id=str(id_single_select.value), 
                                                        base_level=int(base_level_select.value))
            vis_plot.visible = True
        elif vis_select.value == 'icicle':
            vis_plot.object = upload.dataset.generate_icicle(type=int(type_select.value), 
                                                        id=str(id_single_select.value), 
                                                        base_level=int(base_level_select.value))
            vis_plot.visible = True
        elif vis_select.value == 'bar':
            vis_plot.object = upload.dataset.generate_bar(type=int(type_select.value), 
                                                        id=id_list_format.text_list, 
                                                        level=int(base_level_select.value),
                                                        x=x_select.value,
                                                        y=y_select.value,
                                                        log_y=eval(log_select.value))
            vis_plot.visible = True
        elif vis_select.value == 'mean_diff':
            vis_plot.object = upload.dataset.generate_mean_diff(type=int(type_select.value),
                                                                id=id_list_format.text_list,
                                                                level=int(base_level_select.value),
                                                                color=x_select.value)
            vis_plot.visible = True
        elif vis_select.value == 'corr_matrix':
            vis_plot.object = upload.dataset.generate_corr_matrix(type=int(type_select.value),
                                                                  id=id_list_format.text_list,
                                                                  level=int(base_level_select.value))
            vis_plot.visible = True

def upload_image(event):
    ctypes.windll.shcore.SetProcessDpiAwareness(1)
    root = Tk()
    root.withdraw()                                        
    root.call('wm', 'attributes', '.', '-topmost', True) 
    # root.call('tk', 'scaling', 2.0)  
    upload_image.file = filedialog.askopenfilename()
    image_select_text.value = upload_image.file
    region_selector.options = imaging.imaging_read_lookup(imaging.LEVEL_COLUMNS).iloc[:, 0].tolist()
    upload_image.image_is_uploaded = True

    if upload_image.image_is_uploaded:
        image_select_text.visible = True
        image_view_select.visible = True
        image_size_input.visible = True
        image_size_tooltip.visible = True
        region_selector.visible = True

def prev_slice_single(event):
    image_plot_int_input.value = max(0, image_plot_int_input.value - 5)
    image_plot.object = imaging.generate_3d_image(img_path=image_select_text.value, 
                                                        regions=region_selector.value,
                                                        view=image_view_select.value, 
                                                        nrows=image_size_input.value,
                                                        ncols=image_size_input.value,
                                                        slice_n=image_plot_int_input.value)

def next_slice_single(event):
    image_plot_int_input.value = max(0, image_plot_int_input.value + 5)
    image_plot.object = imaging.generate_3d_image(img_path=image_select_text.value, 
                                                        regions=region_selector.value,
                                                        view=image_view_select.value, 
                                                        nrows=image_size_input.value,
                                                        ncols=image_size_input.value,
                                                        slice_n=image_plot_int_input.value)

def prev_slice_multiple(event):
    image_plot_int_input.value = max(0, image_plot_int_input.value - 10)
    image_plot.object = imaging.generate_3d_image(img_path=image_select_text.value, 
                                                        regions=region_selector.value,
                                                        view=image_view_select.value, 
                                                        nrows=image_size_input.value,
                                                        ncols=image_size_input.value,
                                                        slice_n=image_plot_int_input.value)

def next_slice_multiple(event):
    image_plot_int_input.value = max(0, image_plot_int_input.value + 10)
    image_plot.object = imaging.generate_3d_image(img_path=image_select_text.value, 
                                                        regions=region_selector.value,
                                                        view=image_view_select.value, 
                                                        nrows=image_size_input.value,
                                                        ncols=image_size_input.value,
                                                        slice_n=image_plot_int_input.value)
    
def select_slice(event):
    image_plot.object = imaging.generate_3d_image(img_path=image_select_text.value, 
                                                        regions=region_selector.value,
                                                        view=image_view_select.value, 
                                                        nrows=image_size_input.value,
                                                        ncols=image_size_input.value,
                                                        slice_n=image_plot_int_input.value)
    
vis_select.param.watch(check_vis_type, 'value')
id_multi_text.param.watch(id_list_format, 'value')
# slider.param.watch(slide, 'value')
generate_button.on_click(generate)
image_upload_button.on_click(upload_image)
image_plot_left_arrow_single.on_click(prev_slice_single)
image_plot_right_arrow_single.on_click(next_slice_single)
image_plot_left_arrow_multiple.on_click(prev_slice_multiple)
image_plot_right_arrow_multiple.on_click(next_slice_multiple)
image_plot_int_input_button.on_click(select_slice)

image_plot_arrow_row = pn.Row(image_plot_int_input,
                              image_plot_int_input_button,
                              pn.layout.Spacer(width=75),
                              image_plot_left_arrow_multiple,
                              image_plot_left_arrow_single, 
                              image_plot_right_arrow_single,
                              image_plot_right_arrow_multiple,)
image_plot_arrow_row.visible = False
main_header = pn.Row(vis_select, type_select, base_level_select, id_single_select, 
                     id_multi_text, x_select, y_select, log_select, image_upload_button,
                     image_select_text)
image_options = pn.Row(image_view_select, image_size_input, image_size_tooltip)
main_col = pn.Column('# Build Visualizations',
                     main_header,
                     image_options,
                     region_selector,
                     generate_button,
                     pn.layout.Spacer(height=50),
                     image_plot_arrow_row,
                     image_plot,
                     vis_plot)

In [84]:
page = pn.template.BootstrapTemplate(
    title='MRICloudPy',
    sidebar=[side_col],
)

page.main.append(
    pn.Column(main_col)
)

page.show()

Launching server at http://localhost:52426


ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'MessageSent', 'msg_type': 'bokeh_event', 'msg_data': {'type': 'event', 'name': 'button_click', 'values': {'type': 'map', 'entries': [['model', {'id': 'b1db4002-3ff4-4c87-8dbe-a655aabb1f82'}]]}}}]} 
 error: AttributeError("'function' object has no attribute 'is_uploaded'")
Traceback (most recent call last):
  File "c:\Users\Jeremy Telezing\AppData\Local\Programs\Python\Python310\lib\site-packages\bokeh\server\protocol_handler.py", line 97, in handle
    work = await handler(message, connection)
  File "c:\Users\Jeremy Telezing\AppData\Local\Programs\Python\Python310\lib\site-packages\bokeh\server\session.py", line 94, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
  File "c:\Users\Jeremy Telezing\AppData\Local\Programs\Python\Python310\lib\site-packages\bokeh\server\session.py", line 288, in _handle_patch
    message.apply_to_document(self.doc